In [12]:
! pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

wassa_train_csv_path = '/content/drive/My Drive/WASSA_train_all.csv'
wassa_dev_csv_path = '/content/drive/My Drive/WASSA_dev_all.csv'
wassa_train = pd.read_csv(wassa_train_csv_path)
wassa_dev = pd.read_csv(wassa_dev_csv_path)

augmented_path = '/content/drive/My Drive/wassa_all_emb_aug_5.csv'
aug_data = pd.read_csv(augmented_path,encoding='utf-8')
print(wassa_train.shape)
print(aug_data.shape)

(1860, 2)
(9300, 3)


In [16]:
parafrase = aug_data[['paraphrase', 'label']].copy()
parafrase = parafrase.drop_duplicates()
parafrase.rename(columns={"paraphrase": "essay","label":"emotion"}, inplace = True)

wassa_train = wassa_train.append(parafrase,ignore_index=True)
print(wassa_train.shape)

(20460, 2)


<ipython-input-16-787c2a156d3c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wassa_train = wassa_train.append(parafrase,ignore_index=True)


In [17]:
train_df = pd.DataFrame(wassa_train)
train_df.columns = ["text", "labels"]

eval_df = pd.DataFrame(wassa_dev)
eval_df.columns = ["text", "labels"]

In [18]:
import torch
from torch import cuda
import gc

def empty_cache(model, optimizer):
  device = 'cuda' if cuda.is_available() else 'cpu'
  with torch.no_grad():
      torch.cuda.empty_cache()
  gc.collect()

In [19]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, precision_score, recall_score

def f1_multiclass(labels, preds):
  return f1_score(labels, preds, average='macro')

def precision_multiclass(labels,preds):
  return precision_score(labels, preds, average='macro')

def recall_multiclass(labels,preds):
  return recall_score(labels, preds, average='macro')


In [20]:
results = []

reps = 3
for i in range(reps):

  model = ClassificationModel(
    "roberta", 
    "roberta-base",
    num_labels=7, 
    args={'num_train_epochs':2,
          'max_seq_length':256,
          'learning_rate':1e-5,
          'overwrite_output_dir': True, 
          'custom_callback': empty_cache
          }
  )

  model.train_model(train_df)

  result, model_outputs, wrong_predictions = model.eval_model(eval_df, 
                                                              acc=sklearn.metrics.accuracy_score,
                                                              f1=f1_multiclass,
                                                              precision=precision_multiclass,
                                                              recall=recall_multiclass,
                                                              )

  
  results.append({'run': i + 1, 'accuracy': result['acc'], 'f1': result['f1'], 'precision': result['precision'], 'recall': result['recall']})

print('accuracy, f1, precision, recall:')
for i in results:
  print(i['accuracy'], i['f1'], i['precision'], i['recall'])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/20460 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2558 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/2558 [00:00<?, ?it/s]

...

Running Epoch 0 of 2:   0%|          | 0/2558 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/2558 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/34 [00:00<?, ?it/s]

accuracy, f1, precision, recall:
0.674074074074074 0.5705058235510506 0.6111675973569006 0.5732050666495072
0.6592592592592592 0.5825886034854897 0.6138015461218346 0.5769885048368123
0.6296296296296297 0.536475925660066 0.5407198938469018 0.5468746091694013
